In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import datetime
import tqdm
import importlib

In [ ]:
import xarray as xr
import numpy as np
#%matplotlib notebook
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from databroker import catalog

In [ ]:
import bact_analysis
import bact_analysis.bba.calc
import bact_analysis.transverse.calc
import bact_analysis.transverse.distorted_orbit
import bact_analysis.transverse.process
import bact_analysis.transverse.twiss_interpolate
import bact_analysis.utils.preprocess
import bact_analysis_bessyii.bba.preprocess_data
import bact_analysis_bessyii.bba.calc

# BBA Analysis

This notebook should be the center for selecting the required data and presenting the results.
All computation or data rearangement shall be performed in libraries

In [ ]:
datetime.datetime.now().strftime('%Y %m %d %H:%M')

## Data selection, check and load

In [ ]:
catalog_name = 'heavy'

In [ ]:
db = catalog[catalog_name]

For measurement elog entry see http://elog-v2.trs.bessy.de:8080/Machine+Devel.,+Comm./1788

In [ ]:
# uid = '24ff319d-b260-46e6-8356-0dfb08638fb1'
uid = '24ff319d-b260-46e6-8356-0dfb08638fb1'
uid = 'c6c3ad04-7c4c-4a6d-a7e8-91602cfea726'
uid = 'eb89753c-5388-4ebb-a992-fc25b478acd8'

For measurement see elog entry http://elog-v2.trs.bessy.de:8080/Machine+Devel.,+Comm./1971

In [ ]:
uid = 'b226943c-1941-46ce-bc35-0530ea6e276c'
uid = 'e0aef7b8-f57e-4594-9618-70d01aaa65a7'
uid = 'e60215ff-62ea-4d3b-a968-f6b0d9d9ee9d'
uid = 'fa22af2e-0398-41eb-94b9-e9b957ba4f31'

In [ ]:
list(db.keys());

In [ ]:
run = db[uid]

### loading data

In [ ]:
preprocessed_, dt_configuration = bact_analysis_bessyii.bba.preprocess_data.load_and_check_data(run)

In [ ]:
preprocessed_.dt_bpm_waveform_x_rms

### load lattice model

In [ ]:
selected_model_ = xr.load_dataset('bessyii_twiss_thor_scsi.nc')
selected_model_

### Checking consistency between lattice model and digital twin ... for the required elements

Check that all bpm's are in the lattice

In [ ]:
bpm_names = preprocessed_.coords["bpm"]
bpm_names_lc = [name.lower() for name in bpm_names.values]

In [ ]:
bpm_names_check = set(bpm_names_lc)

In [ ]:
bpm_names_check.difference(selected_model_.coords["pos"].values)

Remove first reading ... bpm data not garanteed to be correct

In [ ]:
idx = preprocessed_.dt_cs_setpoint >= 1#
preprocessed = preprocessed_.isel(time=idx)

In [ ]:
preprocessed.dt_cs_setpoint.min();

### Make data selectable per magnet

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)
importlib.reload(bact_analysis.transverse.calc)
importlib.reload(bact_analysis_bessyii.bba.preprocess_data)

In [ ]:
rearranged = xr.concat(
    bact_analysis.utils.preprocess.reorder_by_groups(
        preprocessed,
        preprocessed.groupby(preprocessed.dt_mux_selector_selected),
        reordered_dim="name",
        dim_sel="time",
        new_indices_dim="step",
    ),
    dim="name",
)

In [ ]:
rearranged;

In [ ]:
measurement_vars = dict(
    dt_bpm_waveform_x_pos="x_pos",
    dt_bpm_waveform_y_pos="y_pos",
    dt_bpm_waveform_x_rms="x_rms",
    dt_bpm_waveform_y_rms="y_rms",
    dt_mux_power_converter_setpoint="excitation",
)
redm4proc = (
    rearranged[list(measurement_vars.keys())]
    .rename_vars(**measurement_vars)
    .sel(bpm=bpm_names)
    .rename_dims(bpm="pos")
    .assign_coords(pos=bpm_names_lc)
    .reset_coords(drop=True)
)
# BPM Data are in mm
m2mm = 1./1000.
redm4proc['x_pos'] =  redm4proc.x_pos * m2mm
redm4proc['y_pos'] =  redm4proc.y_pos * m2mm
redm4proc['x_rms'] =  redm4proc.x_rms * m2mm
redm4proc['y_rms'] =  redm4proc.y_rms * m2mm

In [ ]:
redm4proc

## Processing data

### Using model information

This model information is obtained here from the model produced by the digital shadow. 

Should be replaced by a standard lattice from database

In [ ]:
redm4proc.name;

Beam dynamics codes use name labels typically for the start or the end of a model. Tracy uses it for the start. The interpretation here needs it for the middle.

In [ ]:
ds = selected_model_.ds.values
assert( ((ds[1:] - ds[:-1]) >= 0 ).all() )

In [ ]:
quad_twiss_ = bact_analysis.transverse.twiss_interpolate.interpolate_twiss(
    selected_model_, names=[name.lower() for name in redm4proc.name.values]
)
quad_twiss = quad_twiss_.rename_dims(name='pos').assign_coords(pos=quad_twiss_.coords['name'].values).reset_coords(drop=True)
del quad_twiss_
quad_twiss

In [ ]:
selected_model_

Replace these values with the values found before .... 

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)

In [ ]:
n_index = bact_analysis.utils.preprocess.replace_names(list(selected_model_.coords['pos'].values), {name: name + '_s' for name in quad_twiss.coords['pos'].values})
selected_model = xr.concat([selected_model_.assign_coords(pos=n_index), quad_twiss], dim='pos').sortby('ds')
selected_model

### Check if tune advance matches applied current changes

$$
    \Delta Q = \frac{1}{4 \pi} \int_{s_0}^{s_0 + l} \Delta k \beta(s) d(s) 
$$

Measured tune change is 

$$
    \Delta Q_m = \frac{\Delta T}{f_m \cdot n_b}
$$

with $\Delta T$ the measured tune, $f_m$ the main RF frequency and $n_b$ the number of bunches

In [ ]:
quad_name = 'Q4M2D1R'

In [ ]:
for_mag = rearranged.sel(name=quad_name)

In [ ]:
quad_twiss_sel = quad_twiss.sel(pos=quad_name.lower())

In [ ]:
tune_x = for_mag.dt_mr_tune_fb_hor_readback
tune_y = for_mag.dt_mr_tune_fb_vert_readback

In [ ]:
500e6/400 * 0.04 / 1e3

In [ ]:
data = bact_analysis_bessyii.bba.calc.load_calib_data()
data.sel(name=quad_name)

In [ ]:
pos = selected_model_.coords.indexes['pos']
idx = pos.get_loc(quad_name.lower())
idx

In [ ]:
quad_sel = selected_model_.isel(pos=[idx-1, idx, idx+1])
quad_sel.ds - quad_sel.ds.isel(pos=0)

In [ ]:
t_sel = selected_model_.isel(pos=np.arange(-10, 10) + idx)
t_sel
idx = t_sel.coords.indexes['pos'].get_loc(quad_name.lower())
quad_sel = t_sel.isel(pos=[idx-1, idx])
t_sel

In [ ]:
t_sel.ds.values[1:] - t_sel.ds.values[:-1]

In [ ]:
quad_sel

In [ ]:
quad_sel.ds - quad_sel.ds[0]

In [ ]:
quad_twiss_sel.ds

In [ ]:

fig, ax = plt.subplots(1,1, )
line, = ax.plot(t_sel.ds, t_sel.beta.sel(plane='x'), '.-')
ax.plot(quad_sel.ds, quad_sel.beta.sel(plane='x'), 'x-', color=line.get_color(), linewidth=3)
ax.plot(quad_twiss_sel.ds, quad_twiss_sel.beta.sel(plane='x'), '*', color=line.get_color())
line, = ax.plot(t_sel.ds, t_sel.beta.sel(plane='y'), '.-')
ax.plot(quad_sel.ds, quad_sel.beta.sel(plane='y'), 'x-', color=line.get_color(), linewidth=3)
ax.plot(quad_twiss_sel.ds, quad_twiss_sel.beta.sel(plane='y'), '*', color=line.get_color())
ax.set_xlabel("ds [m]")
ax.set_ylabel(r"$\beta_ {x,y}$  [m]")

In [ ]:
def quad_data(name):
    pos = selected_model.coords.indexes['pos']
    idx = pos.get_loc(name)
    assert(idx > 0)
    quad = selected_model.isel(pos=[idx-1,idx])
    return quad

In [ ]:
dtune_y = tune_y -tune_y[0]
dtune_x = tune_x -tune_x[0]
dtune_x.max().values, dtune_y.max().values, (dtune_x.max() /  dtune_y.max()).values

In [ ]:
fig, ax  = plt.subplots(1, 1)
ax.plot(for_mag.coords['step'], dtune_x, 'b.-')
ax.plot(for_mag.coords['step'], dtune_y * -1, 'g.-')

In [ ]:
quad_sel.mean(dim="pos")

In [ ]:
dtune_x_p = bact_analysis_bessyii.bba.calc.predict_tune_change(quad_name, 2, beta=quad_sel.beta.sel(plane='x').mean()) 
dtune_y_p = bact_analysis_bessyii.bba.calc.predict_tune_change(quad_name, 2, beta=quad_sel.beta.sel(plane='y').mean()) 
dtune_y_p, dtune_x_p,  dtune_x_p/ dtune_y_p

In [ ]:
np.absolute(dtune_x).max(), np.absolute(dtune_y).max()

In [ ]:
907/1250

In [ ]:
quad_data('q1m1d1r').beta.sel(plane='y').mean()

In [ ]:
importlib.reload(bact_analysis_bessyii.bba.calc)

In [ ]:
bact_analysis_bessyii.bba.calc.predict_tune_change(quad_name, 2, beta=t_sel.beta.sel(plane='x').mean())

## Process all magnets

In [ ]:
importlib.reload( bact_analysis.transverse.distorted_orbit)
importlib.reload( bact_analysis.transverse.calc)
importlib.reload( bact_analysis.transverse.process)

Calculate the 

In [ ]:
start = datetime.datetime.now()
result = {
    name: item for name, item in 
    tqdm.tqdm(bact_analysis.transverse.process.process_all_gen(selected_model, redm4proc, redm4proc.coords['name'].values, bpm_names=bpm_names_lc, theta=1e-5, use_weights=True), 
              total=len(redm4proc.coords['name']))
}
end  = datetime.datetime.now()
end - start

In [ ]:
quad_names = list(result.keys())
quad_names_lc = [name.lower() for name in quad_names]
ds_quads = selected_model.ds.sel(pos=quad_names_lc).rename(pos='name').assign_coords(name=quad_names)

In [ ]:
rds = bact_analysis.transverse.process.combine_all(result).merge(dict(ds=selected_model.ds, ds_quads=ds_quads)).sortby([ 'ds_quads'])

In [ ]:
rds.orbit.attrs

In [ ]:
rds;

## Check calculated orbits

Compare the calculated fit to the orbit

In [ ]:
redm4proc.sel(name='Q1M1D1R')

In [ ]:
def check_kick_fit(measurement, orbit, parameters):
    """Difference of fit orbit to data
    
    Todo:
          include measurement error
    """
    bpm_names = measurement.coords['pos']
    
    s = parameters.sel(parameter='scaled_angle')
    scale = s.sel(result='value')
    scale_err = s.sel(result='error')
    del s
    
    pars  = parameters.sel(parameter=bpm_names)
    offset =  pars.sel(result='value')
    offset_err = pars.sel(result='error')
    del pars
    
    #print(parameters)
    scaled_orbit = orbit * scale
    scaled_orbit_err = np.absolute(orbit) * scale_err
    
    scaled_orbit_at_bpm = scaled_orbit.sel(pos=bpm_names)
    s_x = scaled_orbit_at_bpm.sel(plane="x") * measurement.excitation
    s_y = scaled_orbit_at_bpm.sel(plane="y") * measurement.excitation
    m_x  = offset.sel(plane="x") - measurement.x_pos
    m_y  = offset.sel(plane="y") - measurement.y_pos
    diff_x = s_x + m_x
    diff_y = s_y + m_y
    del scaled_orbit_at_bpm
    
    so_at_bpm_err = scaled_orbit_err.sel(pos=bpm_names)
    diff_x_err = so_at_bpm_err.sel(plane="x") * measurement.excitation + offset_err.sel(plane="x")
    diff_y_err = so_at_bpm_err.sel(plane="y") * measurement.excitation + offset_err.sel(plane="y")
    del so_at_bpm_err
    
    pos, step = diff_x.dims
    coords = [["x", "y"], ["value", "error", "orbit", "measurement"]]
    dims = ["plane", "result",  pos, step]
    coords += [diff_x.coords[pos], diff_x.coords[step]]
    diff = xr.DataArray(data=[[diff_x, diff_x_err, s_x, m_x], [diff_y, diff_y_err, s_y, m_y]], dims=dims, coords=coords)
    return diff

In [ ]:
def process(name): 
    measurement = redm4proc.sel(name=name)
    orbit = rds.orbit.sel(name=name)
    diff = check_kick_fit(measurement, orbit, rds.fit_params.sel(name=name))
    diff = diff.expand_dims(name=[name])
    return diff

diffs_fit = xr.concat([process(name) for name in rds.coords["name"].values], dim="name")


In [ ]:
np.absolute(diffs_fit.sel(result="value")).max() * 1e6, np.absolute(diffs_fit.sel(result="error")).max() * 1e6

In [ ]:
rng = np.random.default_rng()

In [ ]:
quad_names = [str(name.values) for name in rds.coords["name"]]
quad_names_lc = [name.lower() for name in quad_names]

In [ ]:
y_max =  rds.fit_params.sel(plane="y", result="value", parameter="scaled_angle").argmax().values
y_min =  rds.fit_params.sel(plane="y", result="value", parameter="scaled_angle").argmin().values
x_max =  rds.fit_params.sel(plane="x", result="value", parameter="scaled_angle").argmax().values
x_min =  rds.fit_params.sel(plane="x", result="value", parameter="scaled_angle").argmin().values
y_max, y_min

In [ ]:
list(redm4proc.variables.keys())

In [ ]:
redm4proc.x_pos.sel(name='Q1M1D1R').shape

In [ ]:
plt.plot(
    # redm4proc.x_pos.sel(name='Q1M1D1R', step=4), '-',
    np.sqrt(1./redm4proc.y_rms.sel(name='Q1M1D1R', step=4)), 'x',
)

In [ ]:
name = rng.choice(quad_names)
name = rds.coords["name"][91]
name = rds.coords["name"][x_min]
name = "Q4M2D1R"
name = "Q3M2T2R"
name = "Q4M1T4R"
name = "Q5M2T5R"
t_diff = diffs_fit.sel(name=name)

fig, axes = plt.subplots(2, 2, figsize=[14, 6], sharex=True)
ax_comp, ax_diff = axes
ax_x, ax_y = ax_comp
ax_dx, ax_dy = ax_diff
a_scale = 1e6
err_scale = a_scale * 1
ds = selected_model.ds.sel(pos=t_diff.pos)

for step in t_diff.step:
    t_dI = redm4proc.excitation.sel(name=name, step=step)
    a_scale_ = a_scale * np.sign(t_dI)
    err_scale_ = err_scale #* np.sign(step)
    ax_x.errorbar(
        ds, t_diff.sel(step=step, result="orbit", plane="x") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '-'
    )
    ax_y.errorbar(
        ds, t_diff.sel(step=step, result="orbit", plane="y") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="y")* err_scale_, 
        fmt= '-'
    )
    ax_x.errorbar(
        ds, t_diff.sel(step=step, result="measurement", plane="x") * a_scale_ * -1,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '+'
    )
    ax_y.errorbar(
        ds, t_diff.sel(step=step, result="measurement", plane="y") * a_scale_ * -1,
        yerr=t_diff.sel(step=step, result="error", plane="y") * err_scale_, 
        fmt= '+'
    )
    ax_dx.errorbar(
        ds, t_diff.sel(step=step, result="value", plane="x") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '.'
    )
    ax_dy.errorbar(
        ds, t_diff.sel(step=step, result="value", plane="y") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="y")* err_scale_, 
        fmt= '.'
    )
ax_y.set_xlabel('s [m]')
ax_x.set_ylabel('x [$\mu$m]')
ax_y.set_ylabel('y [$\mu$m]');
ax_dx.set_ylabel('dx [$\mu$m]')
ax_dy.set_ylabel('dy [$\mu$m]');


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = rds.orbit.attrs['theta'] * 1e6
err_scale = a_scale * 5
ds = selected_model.ds.sel(pos=quad_names_lc)
ax_x.errorbar(
    ds, rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='x') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='x') * err_scale, 
    fmt= '.'
)
ax_y.errorbar(
    ds, rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='y') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='y') * err_scale, 
    fmt= '.'
)
ax_y.set_xlabel('s [m]')
ax_x.set_ylabel('x [$\mu rad$]')
ax_y.set_ylabel('y [$\mu$ rad]')


In [ ]:
import bact2_bessyii.magnets
importlib.reload(bact2_bessyii.magnets)

In [ ]:
import bact_analysis_bessyii.bba.calc
importlib.reload(bact_analysis_bessyii.bba.calc)

In [ ]:
calib_quad = bact_analysis_bessyii.bba.calc.load_calib_data()
calib_quad

Impact of muxer:

* the muxer powers an auxilliary coil. This coil has 75 turns
* the main quadrupole coil has 28 turns

In [ ]:
offsets = bact_analysis_bessyii.bba.calc.angles_to_offset_all(rds, names=quad_names, tf_scale=75.0/28.0)
offsets;

In [ ]:
name

In [ ]:
offsets.sel(name=name, result="value")*1e3

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6
ax_x.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_y.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.set_xlabel("s [m]")
ax_x.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")
fig.savefig("axes_offset.pdf")

## Comparison to classic BBA

In [ ]:
import os.path
import pandas as pd
import h5py

In [ ]:
ls BBA_Classic

In [ ]:
t_dir = 'BBA_Classic'

In [ ]:
df_classic  = pd.read_hdf(os.path.join(t_dir, "20210818_quadrupoleOffsetsOldSchoolAnalysis.hdf")).set_index("names")

In [ ]:
ds_classic = xr.Dataset.from_dataframe(df_classic)
ds_classic = ds_classic.rename_dims(names='name').assign_coords(name=[name.strip() for name in ds_classic.names.values]).reset_coords(drop=True)
ds_classic

In [ ]:
ds_classic.sel(name="Q5M2T5R")*1e3

In [ ]:
ds;

In [ ]:
#quad_names = offsets.coords['name'].values
#quad_names_lc = [name.lower() for name in quad_names]
#quad_names;

In [ ]:
set(ds_classic.name.values).difference(quad_names)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6
classic_scale = 1
ax_x.errorbar(
    ds.sel(pos=quad_names_lc),
    offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_x.plot(ds_classic.s.sel(name=quad_names), ds_classic.hor_offset.sel(name=quad_names) * a_scale * classic_scale, '+')
ax_y.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.plot(ds_classic.s.sel(name=quad_names), ds_classic.ver_offset.sel(name=quad_names) * a_scale * classic_scale * -1, '+')
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6
classic_scale = 1e3 * 1
ax_x.errorbar(
    ds.sel(pos=quad_names_lc),
    (offsets.sel(name=quad_names, result="value", plane="x") - ds_classic.hor_offset.sel(name=quad_names)) * a_scale,
     yerr=offsets.sel(name=quad_names, result="error", plane="x")  * a_scale,
    fmt=".-",
)
ax_y.errorbar(
    ds.sel(pos=quad_names_lc),
    (offsets.sel(name=quad_names, result="value", plane="y") - ds_classic.ver_offset.sel(name=quad_names) * -1) * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".-",
)
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.plot(
    ds.sel(pos=quad_names_lc) - ds_classic.s.sel(name=quad_names).values, '-'
    #(ds.sel(pos=quad_names_lc).values -     
)

In [ ]:
np.array([name.lower() for name in quad_names]) == np.array(quad_names_lc)